In [1]:
from active_critic.utils.gym_utils import sample_expert_transitions
import torch as th
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
import gym
from imitation.data.wrappers import RolloutInfoWrapper
from stable_baselines3.common.policies import BaseModel
from typing import Any, Dict, List, Optional, Tuple, Type, TypeVar, Union
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import torch.nn as nn
from active_critic.utils.gym_utils import DummyExtractor
import numpy as np
from active_critic.utils.gym_utils import make_policy_dict
from metaworld.envs import ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE
from gym.wrappers import TimeLimit
from active_critic.utils.pytorch_utils import detokenize, tokenize
from stable_baselines3 import SAC
from active_critic.utils.tboard_graphs import TBoardGraphs

/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
2022-12-12 18:19:13.551696: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [67]:
input = th.zeros([1,1,2,3])
#input = input + 1e-10
input[:,:,:,-1] = 10

In [68]:
label = th.zeros_like(input)
label[:,:,:,-1] = 1

In [69]:
def dist_sm(input, label, scale):
    sm = th.nn.Softmax(dim=-1)
    prob_input = sm(input)
    dist = th.arange(input.shape[-1]).reshape([1,1,1,-1]).repeat([input.shape[0], input.shape[1],input.shape[2], 1])
    arg_label = th.argmax(label, dim=-1)
    arg_dist = dist - arg_label[:, :,:, None]
    arg_dist = arg_dist**2
    arg_dist = arg_dist * scale[None, None,:, None]
    arg_dist = arg_dist * prob_input
    return arg_dist

In [70]:
result = dist_sm(input=input, label=label, scale= th.tensor([0.1,0.1]))

tensor([[[[ 0.,  0., 10.],
          [ 0.,  0., 10.]]]])
tensor([[[[4.5396e-05, 4.5396e-05, 9.9991e-01],
          [4.5396e-05, 4.5396e-05, 9.9991e-01]]]])


In [71]:
result.shape

torch.Size([1, 1, 2, 3])

In [72]:
result

tensor([[[[1.8158e-05, 4.5396e-06, 0.0000e+00],
          [1.8158e-05, 4.5396e-06, 0.0000e+00]]]])